In [1]:
from validphys.loader import FallbackLoader as Loader
from validphys.api import API
from validphys.convolution import predictions
import yaml
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
l = Loader()

In [36]:
fit_names = [f"230310-rs-00{n}{l}" for l in "a" for n in range(2,9)] # different th covmat per alphas

In [37]:
fits = [l.check_fit(f) for f in fit_names]

Resource not in the remote repository: Could not find fit '230310-rs-001a' in remote index fitdata.json


FitNotFound: Could not find fit '230310-rs-001a' in '/home/roy/miniconda3/envs/nnpdf/share/NNPDF/results'. Folder '/home/roy/miniconda3/envs/nnpdf/share/NNPDF/results/230310-rs-001a' not found

In [5]:
indexes = {f: API.fitted_replica_indexes(pdf=f) for f in fit_names}

In [6]:
as_fits = defaultdict(list)
for f in fits:
    th = f.as_input()["theory"]["theoryid"]
    alpha = float(API.theory_info_table(theory_db_id = th).loc["alphas"])
    as_fits[alpha].append(f)
as_fits = dict(as_fits)

In [7]:
# get the indexes that passed postfit for all fits
indexes_list = [API.fitted_replica_indexes(pdf=f) for f in fit_names]
shared_indexes = indexes_list[0]
for reps in indexes_list[1:]:
    shared_indexes = list(set(reps).intersection(shared_indexes))

In [8]:
def get_config(fitname):
    fitpath = API.fit(fit=fitname).path 
    filterpath = fitpath / 'filter.yml'
    with open(filterpath) as f:
        filterfile = yaml.safe_load(f)

    inps_central = dict(
    dataset_inputs={"from_": "fit"},
    fit=fitname,
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
    theoryid=filterfile['theory']['theoryid'],
    pdf=filterfile['theorycovmatconfig']['pdf'],
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
    )
    return inps_central

In [12]:
def measure(fitname):
    config_dict = get_config(fitname)

    # load necessary objects
    C = API.groups_covmat(**config_dict)
    invcov  = np.linalg.inv(C)
    pseudodata = API.read_pdf_pseudodata(**config_dict)
    datasets = API.dataset_inputs(**config_dict)
    pdf = API.pdf(pdf=fitname)
    indexes = API.fitted_replica_indexes(pdf=fitname)

    # calculate theory predictions
    th_preds = []
    for dataset in datasets:
        ds = API.dataset(dataset_input={"dataset": dataset.name, "cfac": dataset.cfac},**config_dict)
        preds = predictions(ds,pdf)
        df = pd.concat({dataset.name: preds}, names=["dataset"])
        th_preds.append(df)
    th_preds = pd.concat(th_preds)

    # reorder to agree with C
    th_preds = th_preds.reindex(C.columns.droplevel(0))

    chi2 = []
    for r in pseudodata:
        # reorder to agree with C
        datarep = pd.DataFrame(r.pseudodata.to_numpy(),index=r.pseudodata.index.droplevel(0))
        datarep = datarep.reindex(C.columns.droplevel(0))
        diff = th_preds[0].to_numpy()-datarep.to_numpy().flatten()
        chi2.append(diff@invcov@diff)

    return chi2


In [13]:
min_values = {}

for alpha, flist in as_fits.items():
    series = []
    for f in flist:
        s = measure(f.name)
        indexes = API.fitted_replica_indexes(pdf=f.name)
        series.append(pd.Series(s, index=indexes))
    min_values[alpha] = pd.DataFrame(series).min()

data = pd.DataFrame(min_values)
data.dropna(axis="index", inplace=True)


LHAPDF 6.5.0 loading /home/roy/miniconda3/envs/nnpdf/share/LHAPDF/230309-rs-008/230309-rs-008_0000.dat
230309-rs-008 PDF set, member #0, version 1
LHAPDF 6.5.0 loading all 215 PDFs in set 230310-rs-008a
230310-rs-008a, version 1; 215 PDF members


In [35]:
mins = {}
abs_mins = {}
for ind, row in data.iterrows():
    a, b, c = np.polyfit(data.columns, row, 2)
    if not np.isnan(b):
        mins[ind] = -b / 2 / a
    abs_mins[ind] = data.columns[np.where(row==row.min())][0]

mins = pd.Series(mins)
abs_mins = pd.Series(abs_mins)

/home/roy/miniconda3/envs/nnpdf/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/roy/miniconda3/envs/nnpdf/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/roy/miniconda3/envs/nnpdf/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/roy/miniconda3/envs/nnpdf/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/roy/miniconda3/envs/nnpdf/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/home